# 招聘与求职信息抓取
此问题分为以下两个部分
- 招聘信息抓取
- 求职信息抓取

In [1]:
import requests

## 招聘信息抓取

根据问题2可以分析出，有以下字段需要抓取：
1. 招聘信息ID
2. 企业名称
3. 招聘岗位
4. 学历要求
5. 岗位需求量
6. 公司类型
7. 薪资待遇
8. 岗位技能
9. 企业工作地点

In [85]:
# 岗位API
job_api = 'https://www.5iai.com/api/enterprise/job/public/es'

In [80]:
# 设定Get请求参数
job_params = {
    'pageSize': '2000',
    'pageNumber': 1
}

In [81]:
# 发送GET请求
job_response = requests.get(job_api, params=job_params)

In [82]:
# 将响应内容转化为字典
job_dict = job_response.json()

In [83]:
# 获取岗位信息列表
job_list = job_dict['data']['content']
# 查看岗位数量
print('共获取%d个岗位' % len(job_list))

共获取1574个岗位


### 招聘信息ID

In [22]:
job_id = [i['id'] for i in job_list]

### 企业名称

In [29]:
job_name = [i['enterpriseExtInfo']['shortName'] for i in job_list]

### 招聘岗位

In [30]:
job_positionName = [i['positionName'] for i in job_list]

### 学历要求

In [31]:
job_educationalRequirements = [i['educationalRequirements'] for i in job_list]

In [90]:
job_educationalRequirements

2

In [105]:
# 在此出学历要求为数值，经过观察发现在网页的加载过程中会去请求一个educational_requirements的API接口，观察发现其中有上述数值对应的具体学历名称
# 后续在使用时可调用以下字典还原为具体名称
education_dict = {i['value']: i['label'] for i in requests.get('https://www.5iai.com/api/dict/data/public/list/tree?dictType=education_requirement').json()['data']}
education_dict

{'0': '不限', '1': '技工', '2': '大专', '3': '本科', '4': '硕士', '5': '博士'}

### 岗位需求量

0代表人数不限

In [33]:
job_count = [i['count'] for i in job_list]

### 公司类型

In [34]:
job_econKind = [i['enterpriseExtInfo']['econKind'] for i in job_list]

### 薪资待遇

薪资待遇分为最高薪资和最低薪资

In [36]:
job_minimumWage = [i['minimumWage'] for i in job_list]
job_maximumWage =  [i['maximumWage'] for i in job_list]

### 岗位技能、企业工作地点

岗位技能和工作地点在二级页面

In [71]:
# 二级页面详细信息API
job_secondPage_api = 'https://www.5iai.com/api/enterprise/job/public'
job_skillsList = []
job_address = []
job_Requiredments = []
for i in range(len(job_id)):
    job_secondPage_params = {
        'id': job_id[i]
    }
    # 访问二级页面
    job_secondPage_response = requests.get(job_secondPage_api, params=job_secondPage_params)
    job_secondPage_dict = job_secondPage_response.json()
    if 'data' in job_secondPage_dict.keys():
        # 获取岗位技能
        job_skillsList.append([i['labelName'] for i in job_secondPage_dict['data']['skillsList']])
        # 获取工作地点
        job_address.append(job_secondPage_dict['data']['enterpriseAddress']['detailedAddress'])
        # 获取岗位职责
        job_Requiredments.append(job_secondPage_dict['data']['jobRequiredments'])
    else:
        # 如果没有岗位详细信息，即二级页面为空
        job_skillsList.append('')
        job_address.append('')
        job_Requiredments.append('')

## 循环将招聘信息数据存储到数据库中

In [2]:
import pymysql

In [3]:
# 连接数据库
conn = pymysql.connect(host='localhost',
                      port=3306,
                      database='tipdm_cup',
                      user='root',
                      passwd='123456')

In [4]:
# 创建游标
cursor = conn.cursor()

In [87]:
# 二级页面API
job_secondPage_api = 'https://www.5iai.com/api/enterprise/job/public'

In [88]:
n = 1
for i in job_list:
    job_id = i['id']
    job_name = i['enterpriseExtInfo']['shortName']
    job_positionName = i['positionName']
    job_educationalRequirements = i['educationalRequirements']
    job_count = i['count']
    job_econKind = i['enterpriseExtInfo']['econKind']
    job_minimumWage = i['minimumWage']
    job_maximumWage = i['maximumWage']
    
    job_secondPage_params = {
        'id': job_id
    }
    # 访问二级页面
    job_secondPage_response = requests.get(job_secondPage_api, params=job_secondPage_params)
    job_secondPage_dict = job_secondPage_response.json()
    if 'data' in job_secondPage_dict.keys():
        # 获取岗位技能
        job_skillsList = ','.join([i['labelName'] for i in job_secondPage_dict['data']['skillsList']])
        # 获取工作地点
        job_address = job_secondPage_dict['data']['enterpriseAddress']['detailedAddress']
        # 获取岗位职责
        job_Requiredments = job_secondPage_dict['data']['jobRequiredments']
    else:
        # 如果没有岗位详细信息，即二级页面为空
        job_skillsList = ''
        job_address = ''
        job_Requiredments = ''
    
    job_sql = f"insert into job values(0, '{job_id}', '{job_name}', '{job_positionName}', '{job_educationalRequirements}', '{job_count}', '{job_econKind}', '{job_minimumWage}', '{job_maximumWage}', '{job_skillsList}', '{job_address}', '{job_Requiredments}')"
    cursor.execute(job_sql)
    # 提交数据库
    conn.commit()
    print(f'岗位{n}: {job_id}已保存！')
    n += 1

岗位1: 1631112859985510400已保存！


In [ ]:
# 关闭游标
cursor.close()
# 关闭数据库
conn.close()

## 求职信息抓取

根据问题2可以分析出，有以下字段需要抓取：
1. 求职信息ID
2. 姓名
3. 学历
4. 工作经验
5. 预期岗位
6. 预期薪资
7. 预期工作地点
8. 知识储备/技能
9. 工作经历
10. 项目经历
11. 竞赛经历
12. 证书

In [2]:
# 求职API
user_api = 'https://www.5iai.com/api/resume/baseInfo/public/es'

In [3]:
# 设定Get请求参数
user_params = {
    'pageSize': 99999,
    'pageNumber': 1
}

In [4]:
# 发送GET请求
user_response = requests.get(user_api, params=user_params)

In [5]:
# 将响应内容转化为字典
user_dict = user_response.json()

In [6]:
# 获取求职者信息列表
user_list =  user_dict['data']['content']

### 求职信息ID

In [213]:
user_id = [i['id'] for i in user_list]

### 姓名

In [8]:
user_name = [i['username'] for i in user_list]

### 工作经验

In [199]:
user_exp = [i['exp'] for i in user_list]

### 预期岗位

In [10]:
user_expectPosition = [i['expectPosition'] for i in user_list]

### 预期薪资

In [11]:
user_willSalaryEnd = [i['willSalaryEnd'] for i in user_list]
user_willSalaryStart = [i['willSalaryStart'] for i in user_list]

### 二级页面数据抓取

学历、预期工作地点、知识储备/技能、工作经历、项目经历、竞赛经历、证书情况等信息在二级页面

In [200]:
user_url = 'https://www.5iai.com/api/resume/baseInfo/public/1632947816114552832'

'https://www.5iai.com/api/resume/baseInfo/public/1632947816114552832'

In [202]:
user_Secoond_response = requests.get(user_url)
user_Secoond_dict = user_Secoond_response.json()

#### 学历信息

In [205]:
','.join([i['school']+'_'+i['speciality']+'_'+i['educationalBackground']for i in user_Secoond_dict['data']['educationExperienceList']])

'广东第二师范学院_统计学本科'

#### 预期工作地点

In [168]:
user_Secoond_dict['data']['city']

'["广东省","深圳市","南山区"]'

#### 知识储备/技能

In [191]:
','.join([i['name']+'_'+i['level'] for i in user_Secoond_dict['data']['skillList']])

'python_MASTER,数据分析_SKILLED,数据建模（机器学习）_SKILLED,网络爬虫_GOOD'

#### 工作经历

In [196]:
','.join([i['company'] + '_' + i['positionName'] for i in user_Secoond_dict['data']['workExperienceList']])

'广东泰迪智能科技股份有限公司_助教'

#### 项目经历

In [193]:
','.join([i['roleName'] for i in user_Secoond_dict['data']['projectExperienceList']])

'数据分析'

#### 竞赛经历

In [183]:
','.join([i['name']+'_'+i['level'] for i in user_Secoond_dict['data']['competitionExperienceList']])

'第四届泰迪杯数据分析技能赛_一等奖,“第七届中国国际“互联网+大学生创新创业大赛广东省分赛_铜牌,计算机程序设计员(Python编程)竞赛全国总决赛_三等奖,第九届泰迪杯数据挖掘挑战赛_三等奖'

#### 证书情况

In [185]:
','.join([i['name'] for i in user_Secoond_dict['data']['certList']])

'1+X中级数据分析师'

#### 电子版简历上传情况

In [187]:
','.join([i['name']+'_'+i['file'] for i in user_Secoond_dict['data']['attachmentList']])

'李晓简历_/userjcq1mj/1677825079339/1677825079339.pdf'

## 循环将求职信息数据存储到数据库

In [68]:
import pymysql

In [69]:
# 连接数据库
conn = pymysql.connect(host='localhost',
                      port=3306,
                      database='tipdm_cup',
                      user='root',
                      passwd='123456')

In [70]:
# 创建游标
cursor = conn.cursor()

In [214]:
n = 1
for i in user_list:
    user_id = i['id']
    user_name = i['username']
    user_exp = i['exp']
    user_expectPosition = i['expectPosition']
    user_willSalaryEnd = i['willSalaryEnd']
    user_willSalaryStart = i['willSalaryStart']
    
    # 访问二级页面
    user_url = 'https://www.5iai.com/api/resume/baseInfo/public/' + user_id
    user_Secoond_response = requests.get(user_url)
    user_Secoond_dict = user_Secoond_response.json()

    if 'data' in user_Secoond_dict.keys():
        # 获取学历信息
        user_educationExperienceList = ','.join([i['school']+'_'+i['speciality']+'_'+i['educationalBackground']for i in user_Secoond_dict['data']['educationExperienceList']])
        # 获取预期工作地点
        user_city = user_Secoond_dict['data']['city']
        # 获取知识储备/技能
        user_skillList = ','.join([i['name']+'_'+i['level'] for i in user_Secoond_dict['data']['skillList']])
        # 获取工作经历
        user_workExperienceList = ','.join([i['company'] + '_' + i['positionName'] for i in user_Secoond_dict['data']['workExperienceList']])
        # 获取项目经历
        user_projectExperienceList = ','.join([i['roleName'] for i in user_Secoond_dict['data']['projectExperienceList']])
        # 获取竞赛经历
        user_competitionExperienceList = ','.join([i['name']+'_'+i['level'] for i in user_Secoond_dict['data']['competitionExperienceList']])
        # 获取证书情况
        user_certList = ','.join([i['name'] for i in user_Secoond_dict['data']['certList']])
        # 获取电子版简历上传情况
        user_attachmentList = ','.join([i['name']+'_'+i['file'] for i in user_Secoond_dict['data']['attachmentList']])
    else:
        # 如果没有岗位详细信息，即二级页面为空
        user_educationExperienceList = ''
        user_city = ''
        user_skillList = ''
        user_workExperienceList = ""
        user_projectExperienceList = ""
        user_competitionExperienceList = ""
        user_certList = ""
        user_attachmentList = ""
    
    user_sql = f"insert into user values(0, '{user_id}', '{user_name}', '{user_exp}', '{user_expectPosition}', '{user_willSalaryEnd}', '{user_willSalaryStart}', '{user_educationExperienceList}', '{user_city}', '{user_skillList}', '{user_workExperienceList}', '{user_projectExperienceList}', '{user_competitionExperienceList}', '{user_certList}', '{user_attachmentList}');"
    cursor.execute(user_sql)
    # 提交数据库
    conn.commit()
    print(f'求职信息{n}: {user_id}已保存！')
    n += 1

求职信息1: 7536434415353392660已保存！
求职信息2: 7536434428238294548已保存！
求职信息3: 7536434441123196436已保存！
求职信息4: 7536434449713131028已保存！
求职信息5: 7536434462598032916已保存！
求职信息6: 7536434466893000212已保存！
求职信息7: 7536434475482934804已保存！
求职信息8: 7536434484072869396已保存！
求职信息9: 7536434496957771284已保存！
求职信息10: 7536434509842673172已保存！
求职信息11: 7536434518432607764已保存！
求职信息12: 7536434531317509652已保存！
求职信息13: 7536434544202411540已保存！
求职信息14: 7536434552792346132已保存！
求职信息15: 7536434565677248020已保存！
求职信息16: 7536434578562149908已保存！
求职信息17: 7536434587152084500已保存！
求职信息18: 7536477283421974036已保存！
求职信息19: 7536477296306875924已保存！
求职信息20: 7536477309191777812已保存！
求职信息21: 7536477317781712404已保存！
求职信息22: 7536477330666614292已保存！
求职信息23: 7536477334961581588已保存！
求职信息24: 7536477347846483476已保存！
求职信息25: 7536477360731385364已保存！
求职信息26: 7536477373616287252已保存！
求职信息27: 7536477382206221844已保存！
求职信息28: 7536477395091123732已保存！
求职信息29: 7536477407976025620已保存！
求职信息30: 7536477416565960212已保存！
求职信息31: 7536477429450862100已保存！
求职信息32: 753647744

求职信息253: 7536906758676737556已保存！
求职信息254: 7536906771561639444已保存！
求职信息255: 7536906784446541332已保存！
求职信息256: 7536906793036475924已保存！
求职信息257: 7536949467831528980已保存！
求职信息258: 7536949472126496276已保存！
求职信息259: 7536949480716430868已保存！
求职信息260: 7536949493601332756已保存！
求职信息261: 7536949506486234644已保存！
求职信息262: 7536949519371136532已保存！
求职信息263: 7536949532256038420已保存！
求职信息264: 7536949540845973012已保存！
求职信息265: 7536949553730874900已保存！
求职信息266: 7536949566615776788已保存！
求职信息267: 7536949579500678676已保存！
求职信息268: 7536949588090613268已保存！
求职信息269: 7536949600975515156已保存！
求职信息270: 7536949613860417044已保存！
求职信息271: 7536949622450351636已保存！
求职信息272: 7536949635335253524已保存！
求职信息273: 7536949652515122708已保存！
求职信息274: 7536949665400024596已保存！
求职信息275: 7536949673989959188已保存！
求职信息276: 7536949686874861076已保存！
求职信息277: 7536949699759762964已保存！
求职信息278: 7536992404619587092已保存！
求职信息279: 7536992417504488980已保存！
求职信息280: 7536992430389390868已保存！
求职信息281: 7536992438979325460已保存！
求职信息282: 7536992451864227348已保存！
求职信息283: 7

求职信息503: 7537379479957204500已保存！
求职信息504: 7537379488547139092已保存！
求职信息505: 7537379492842106388已保存！
求职信息506: 7537379501432040980已保存！
求职信息507: 7537379514316942868已保存！
求职信息508: 7537379522906877460已保存！
求职信息509: 7537422300781145620已保存！
求职信息510: 7537422313666047508已保存！
求职信息511: 7537422322255982100已保存！
求职信息512: 7537422335140883988已保存！
求职信息513: 7537422348025785876已保存！
求职信息514: 7537422360910687764已保存！
求职信息515: 7537422369500622356已保存！
求职信息516: 7537422382385524244已保存！
求职信息517: 7537422395270426132已保存！
求职信息518: 7537422403860360724已保存！
求职信息519: 7537422416745262612已保存！
求职信息520: 7537422429630164500已保存！
求职信息521: 7537422446810033684已保存！
求职信息522: 7537422459694935572已保存！
求职信息523: 7537422468284870164已保存！
求职信息524: 7537465138784955924已保存！
求职信息525: 7537465151669857812已保存！
求职信息526: 7537465160259792404已保存！
求职信息527: 7537465173144694292已保存！
求职信息528: 7537465186029596180已保存！
求职信息529: 7537465194619530772已保存！
求职信息530: 7537465207504432660已保存！
求职信息531: 7537465220389334548已保存！
求职信息532: 7537465228979269140已保存！
求职信息533: 7

求职信息753: 7537893991269461524已保存！
求职信息754: 7537894004154363412已保存！
求职信息755: 7537894017039265300已保存！
求职信息756: 7537894029924167188已保存！
求职信息757: 7537894038514101780已保存！
求职信息758: 7537894051399003668已保存！
求职信息759: 7537894064283905556已保存！
求职信息760: 7537936747668893204已保存！
求职信息761: 7537936760553795092已保存！
求职信息762: 7537936769143729684已保存！
求职信息763: 7537936782028631572已保存！
求职信息764: 7537936794913533460已保存！
求职信息765: 7537936803503468052已保存！
求职信息766: 7537936812093402644已保存！
求职信息767: 7537936824978304532已保存！
求职信息768: 7537936837863206420已保存！
求职信息769: 7537936850748108308已保存！
求职信息770: 7537936859338042900已保存！
求职信息771: 7537936872222944788已保存！
求职信息772: 7537936885107846676已保存！
求职信息773: 7537936893697781268已保存！
求职信息774: 7537936906582683156已保存！
求职信息775: 7537936919467585044已保存！
求职信息776: 7537936928057519636已保存！
求职信息777: 7537936940942421524已保存！
求职信息778: 7537936953827323412已保存！
求职信息779: 7537936966712225300已保存！
求职信息780: 7537936975302159892已保存！
求职信息781: 7537936979597127188已保存！
求职信息782: 7537936988187061780已保存！
求职信息783: 7

求职信息1003: 7538366158499147284已保存！
求职信息1004: 7538366171384049172已保存！
求职信息1005: 7538366179973983764已保存！
求职信息1006: 7538366188563918356已保存！
求职信息1007: 7538366197153852948已保存！
求职信息1008: 7538366210038754836已保存！
求职信息1009: 7538366218628689428已保存！
求职信息1010: 7538366227218624020已保存！
求职信息1011: 7538408936373415444已保存！
求职信息1012: 7538408944963350036已保存！
求职信息1013: 7538408953553284628已保存！
求职信息1014: 7538408962143219220已保存！
求职信息1015: 7538408970733153812已保存！
求职信息1016: 7538408983618055700已保存！
求职信息1017: 7538408992207990292已保存！
求职信息1018: 7538409000797924884已保存！
求职信息1019: 7538409009387859476已保存！
求职信息1020: 7538409017977794068已保存！
求职信息1021: 7538409022272761364已保存！
求职信息1022: 7538409030862695956已保存！
求职信息1023: 7538409039452630548已保存！
求职信息1024: 7538409052337532436已保存！
求职信息1025: 7538409060927467028已保存！
求职信息1026: 7538409069517401620已保存！
求职信息1027: 7538409078107336212已保存！
求职信息1028: 7538409086697270804已保存！
求职信息1029: 7538409095287205396已保存！
求职信息1030: 7538409103877139988已保存！
求职信息1031: 7538409116762041876已保存！
求职信息1032: 7538

求职信息1244: 7538753083512909332已保存！
求职信息1245: 7538795371760905748已保存！
求职信息1246: 7538795380350840340已保存！
求职信息1247: 7538795393235742228已保存！
求职信息1248: 7538795401825676820已保存！
求职信息1249: 7538795410415611412已保存！
求职信息1250: 7538795419005546004已保存！
求职信息1251: 7538795427595480596已保存！
求职信息1252: 7538795436185415188已保存！
求职信息1253: 7538795449070317076已保存！
求职信息1254: 7538795457660251668已保存！
求职信息1255: 7538795466250186260已保存！
求职信息1256: 7538795474840120852已保存！
求职信息1257: 7538795483430055444已保存！
求职信息1258: 7538795492019990036已保存！
求职信息1259: 7538795504904891924已保存！
求职信息1260: 7538795509199859220已保存！
求职信息1261: 7538838218354650644已保存！
求职信息1262: 7538838226944585236已保存！
求职信息1263: 7538838235534519828已保存！
求职信息1264: 7538838248419421716已保存！
求职信息1265: 7538838257009356308已保存！
求职信息1266: 7538838265599290900已保存！
求职信息1267: 7538838274189225492已保存！
求职信息1268: 7538838282779160084已保存！
求职信息1269: 7538838291369094676已保存！
求职信息1270: 7538838299959029268已保存！
求职信息1271: 7538838308548963860已保存！
求职信息1272: 7538838321433865748已保存！
求职信息1273: 7538

求职信息1486: 7539138900425109012已保存！
求职信息1487: 7539138909015043604已保存！
求职信息1488: 7539181575220162068已保存！
求职信息1489: 7539181583810096660已保存！
求职信息1490: 7539181592400031252已保存！
求职信息1491: 7539181605284933140已保存！
求职信息1492: 7539181613874867732已保存！
求职信息1493: 7539181622464802324已保存！
求职信息1494: 7539181631054736916已保存！
求职信息1495: 7539181639644671508已保存！
求职信息1496: 7539181648234606100已保存！
求职信息1497: 7539181656824540692已保存！
求职信息1498: 7539181665414475284已保存！
求职信息1499: 7539181669709442580已保存！
求职信息1500: 7539181678299377172已保存！
求职信息1501: 7539181686889311764已保存！
求职信息1502: 7539181695479246356已保存！
求职信息1503: 7539181708364148244已保存！
求职信息1504: 7539181716954082836已保存！
求职信息1505: 7539181725544017428已保存！
求职信息1506: 7539181734133952020已保存！
求职信息1507: 7539181742723886612已保存！
求职信息1508: 7539181751313821204已保存！
求职信息1509: 7539181759903755796已保存！
求职信息1510: 7539181772788657684已保存！
求职信息1511: 7539181781378592276已保存！
求职信息1512: 7539181789968526868已保存！
求职信息1513: 7539181798558461460已保存！
求职信息1514: 7539181807148396052已保存！
求职信息1515: 7539

求职信息1727: 7539525662230113812已保存！
求职信息1728: 7539525670820048404已保存！
求职信息1729: 7539525683704950292已保存！
求职信息1730: 7539525692294884884已保存！
求职信息1731: 7539525700884819476已保存！
求职信息1732: 7539525709474754068已保存！
求职信息1733: 7539525718064688660已保存！
求职信息1734: 7539525726654623252已保存！
求职信息1735: 7539525735244557844已保存！
求职信息1736: 7539525743834492436已保存！
求职信息1737: 7539525752424427028已保存！
求职信息1738: 7539525761014361620已保存！
求职信息1739: 7539525765309328916已保存！
求职信息1740: 7539525773899263508已保存！
求职信息1741: 7539525782489198100已保存！
求职信息1742: 7539525795374099988已保存！
求职信息1743: 7539568470169153044已保存！
求职信息1744: 7539568478759087636已保存！
求职信息1745: 7539568487349022228已保存！
求职信息1746: 7539568495938956820已保存！
求职信息1747: 7539568504528891412已保存！
求职信息1748: 7539568513118826004已保存！
求职信息1749: 7539568521708760596已保存！
求职信息1750: 7539568534593662484已保存！
求职信息1751: 7539568543183597076已保存！
求职信息1752: 7539568551773531668已保存！
求职信息1753: 7539568560363466260已保存！
求职信息1754: 7539568568953400852已保存！
求职信息1755: 7539568577543335444已保存！
求职信息1756: 7539

求职信息1968: 7539868413505236500已保存！
求职信息1969: 7539868417800203796已保存！
求职信息1970: 7539868426390138388已保存！
求职信息1971: 7539868434980072980已保存！
求职信息1972: 7539868447864974868已保存！
求职信息1973: 7539868456454909460已保存！
求职信息1974: 7539868465044844052已保存！
求职信息1975: 7539868473634778644已保存！
求职信息1976: 7539868486519680532已保存！
求职信息1977: 7539868495109615124已保存！
求职信息1978: 7539868503699549716已保存！
求职信息1979: 7539868512289484308已保存！
求职信息1980: 7539868520879418900已保存！
求职信息1981: 7539868529469353492已保存！
求职信息1982: 7539868538059288084已保存！
求职信息1983: 7539868546649222676已保存！
求职信息1984: 7539868555239157268已保存！
求职信息1985: 7539868568124059156已保存！
求职信息1986: 7539868576713993748已保存！
求职信息1987: 7539868585303928340已保存！
求职信息1988: 7539868589598895636已保存！
求职信息1989: 7539868593893862932已保存！
求职信息1990: 7539868602483797524已保存！
求职信息1991: 7539868611073732116已保存！
求职信息1992: 7539868619663666708已保存！
求职信息1993: 7539911358883229204已保存！
求职信息1994: 7539911371768131092已保存！
求职信息1995: 7539911380358065684已保存！
求职信息1996: 7539911388948000276已保存！
求职信息1997: 7539

# 招聘与求职信息分析

## 招聘数据分析

In [2]:
# 查看存储的数据结果


## 求职数据分析

# 构建岗位匹配度和求职者满意度的模型

# 招聘求职双向推荐系统